# 10/27 Take Home Problems

## 1050. Actors and Directors Who Cooperated At Least Three Times

In [ ]:
SELECT
    actor_id, director_id
FROM
    ActorDirector
GROUP BY
    actor_id, director_id
HAVING
    count(timestamp) >= 3;

In [ ]:
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    actor_director = actor_director.groupby(["actor_id", "director_id"])["timestamp"].size().reset_index()

    return actor_director[actor_director.timestamp >= 3].iloc[:, [0, 1]]

## 1667. Fix Names in a Table

In [ ]:
SELECT 
    user_id, CONCAT(UPPER(SUBSTR(name, 1, 1)), LOWER(SUBSTR(name, 2))) AS name 
FROM
    Users 
ORDER BY
    user_id;

In [ ]:
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users["name"] = users["name"].str.capitalize()
    result_df = users.sort_values(by="user_id", ascending=True)
    
    return result_df

## 175. Combine Two Tables

In [ ]:
SELECT
    P.firstName, P.lastName, A.city, A.state
FROM
    Person P LEFT JOIN Address A
ON
    P.personId = A.personId

In [ ]:
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(left=person, right=address, how="left", on="personId")[["firstName", "lastName", "city", "state"]]

## 176. Second Highest Salary

In [ ]:
SELECT (
    SELECT
        DISTINCT Salary
    FROM
        Employee ORDER BY Salary DESC
    LIMIT 1
    OFFSET 1
) as SecondHighestSalary;

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    unique_salaries = employee["salary"].drop_duplicates()
    second_highest = unique_salaries.nlargest(2).iloc[-1] if len(unique_salaries) >= 2 else None

    if second_highest is None:
        return pd.DataFrame({"SecondHighestSalary": [None]})

    result_df = pd.DataFrame({"SecondHighestSalary": [second_highest]})

    return result_df

## 1327. List the Products Ordered in a Period

In [ ]:
SELECT
    p.product_name AS product_name, sum(o.unit) AS unit
FROM
    Products p
JOIN
    Orders o
USING
    (product_id)
WHERE
    YEAR(o.order_date)="2020" AND MONTH(o.order_date)="02"
GROUP BY
    p.product_id
HAVING
    SUM(o.unit)>=100

In [ ]:
import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    february_2020_orders = orders[orders["order_date"].dt.year == 2020]
    february_2020_orders = february_2020_orders[orders["order_date"].dt.month == 2]

    merged = pd.merge(february_2020_orders, products, on="product_id")
    result = merged.groupby("product_name")["unit"].sum().reset_index()
    result = result[result["unit"] >= 100]

    return result[["product_name", "unit"]]

## 1378. Replace Employee ID With The Unique Identifier

In [ ]:
SELECT
    eu.unique_id as unique_id, e.name as name
FROM
    Employees e
LEFT JOIN
    EmployeeUNI eu
ON
    e.id = eu.id

In [ ]:
import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    merged = employees.merge(employee_uni, on="id", how="left")
    result = merged[["unique_id","name"]]
    
    return result

## 550. Game Play Analysis IV

In [ ]:
SELECT
    ROUND(COUNT(DISTINCT player_id) / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM
    Activity
WHERE
    (player_id, DATE_SUB(event_date, INTERVAL 1 DAY))
IN (
    SELECT player_id, MIN(event_date) AS first_login FROM Activity GROUP BY player_id
)

In [ ]:
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
      activity["first"] = activity.groupby("player_id").event_date.transform(min)
      activity_2nd_day = activity.loc[activity["first"] + pd.DateOffset(1) == activity["event_date"]]
      
      return pd.DataFrame({"fraction":[round(len(activity_2nd_day) / activity.player_id.nunique(),2)]})

## 1075. Project Employees I

In [ ]:
SELECT
    p.project_id, ROUND(AVG(e.experience_years),2) AS average_years
FROM
    Project p 
LEFT JOIN
    Employee e
ON
    p.employee_id = e.employee_id
GROUP BY
    p.project_id

In [ ]:
import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    return (project.merge(employee)
                    .rename(columns = {"experience_years":"average_years"})
                    .groupby("project_id")["average_years"].mean()
                    .round(2).reset_index())

## 185. Department Top Three Salaries

In [ ]:
SELECT
    d.name as department , e1.name as employee, e1.salary as Salary
FROM
    Employee e1
JOIN
    Department d on e1.DepartmentId = d.Id
WHERE
    3 > (SELECT
            count(DISTINCT (e2.Salary))
        FROM
            Employee e2
        WHERE
            e2.Salary > e1.Salary AND e1.DepartmentId = e2.DepartmentId)

In [ ]:
import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    employee.columns = ["id", "Employee", "Salary", "d_id"]
    department.columns = ["d_id", "Department"]
    employee["rnk"] = employee.groupby("d_id")[["Salary"]].rank(method="dense", ascending=False)

    return employee[employee.rnk <= 3].merge(department, on="d_id").iloc[:,[5,1,2]]